In [ ]:
!pip install tensorflow

In [ ]:
import tensorflow as tf
import numpy as np

In [ ]:
def load_data(file_path):
  with open(file_path, 'r') as f:
    data = f.read()
  return data

file_path = 'hp_1.txt'
text = load_data(file_path).lower()

In [ ]:
text

In [ ]:
#Tokenization

from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(oov_token='<OOV>')
tokenizer.fit_on_texts([text])

total_words = len(tokenizer.word_index) + 1

print(tokenizer.word_index)

print(total_words)

In [ ]:
input_sequences = []
tokens = tokenizer.texts_to_sequences([text])[0]

print(tokens)


In [ ]:
sq_len = 50

for i in range(sq_len, len(tokens)):
  seq = tokens[i-sq_len:i+1]
  input_sequences.append(seq)

input_sequences = np.array(input_sequences)

In [ ]:
len(input_sequences[0])
print(input_sequences[0])

In [ ]:
from tensorflow.keras.utils import pad_sequences, to_categorical

input_sequences = np.array(pad_sequences(input_sequences, maxlen=sq_len+1, padding='pre'))
X = input_sequences[:,:-1] # value till last
Y = input_sequences[:,-1] # last value

In [ ]:
Y = tf.keras.utils.to_categorical(Y, num_classes=total_words)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense, LSTM

In [ ]:
model = Sequential([
    Embedding(input_dim = total_words, output_dim=64, input_length=sq_len),
    SimpleRNN(254),
    Dense(256, activation='relu'),
    Dense(total_words, activation='softmax')
])

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history_model = model.fit(X,Y, epochs=10, batch_size=128 )

In [ ]:
model.summary()

In [ ]:
def generate_text(seed_text, next_words):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=sq_len, padding='pre')
        predicted_probs = model.predict(token_list, verbose=0)
        predicted_index = np.argmax(predicted_probs)
        predicted_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted_index:
                predicted_word = word
                break
        seed_text += " " + predicted_word
    return seed_text

# Example of using the generate_text function
print(generate_text("harry looked at", next_words=25))

In [ ]:
model1 = Sequential([
    Embedding(input_dim=total_words, output_dim=100, input_length=sq_len),
    LSTM(256, return_sequences=True),
    LSTM(256),
    Dense(total_words, activation='softmax')
])

In [ ]:
model1.compile( optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'] )

In [ ]:
history_model1 = model1.fit(X, Y, epochs=10, batch_size=128)

In [ ]:
model1.summary()

In [ ]:
def generate_text(seed_text, next_words):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=sq_len, padding='pre')
        predicted_probs = model1.predict(token_list, verbose=0)
        predicted_index = np.argmax(predicted_probs)
        predicted_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted_index:
                predicted_word = word
                break
        seed_text += " " + predicted_word
    return seed_text

# Example of using the generate_text function
print(generate_text("harry looked at", next_words=25))

In [ ]:
loss_rnn, test_acc_rnn = model.evaluate(X, Y, verbose=0)
loss_lstm, test_acc_lstm = model1.evaluate(X, Y, verbose=0)

In [ ]:
import matplotlib.pyplot as plt

print("\n--- Model Comparison ---")
print(f"SimpleRNN (model) Test Accuracy: {test_acc_rnn:.4f}")
print(f"LSTM (model1) Test Accuracy:      {test_acc_lstm:.4f}")



plt.figure(figsize=(12, 6))
plt.plot(history_model.history['val_accuracy'], label='SimpleRNN (model) Validation Accuracy', marker='o')
plt.plot(history_model1.history['val_accuracy'], label='LSTM (model1) Validation Accuracy', marker='x')
plt.title('SimpleRNN vs. LSTM Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.show()